In [2]:
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
from utils import (
    DEGREES_PER_PIXEL,
    TIMESTAMP_IDENT,
    X_PIXELS,
    Y_PIXELS,
    Events,
    extract_gaze_data_between_timestamps_proper,
    get_participant_dominant_eye,
)
import json


from velocityThreshold import detect_fix_ivt, find_sacc_from_fix

# from scipy import ttest_ind

EYE_TRACKER_FOLDER = "eye_tracker_data/"
GAZE_DATA = []
GAZE_DATA_BOOK = []
GAZE_DATA_PAGE = []

# participants that were told that they can change the settings beforehand
participant_ids = [9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]
subquery = Events.select().where(Events.participant_id.in_(participant_ids))
avg_saccades = {_id: [] for _id in participant_ids}

print(f"Mean , std , difference")

for participant in participant_ids:
    books = subquery.where(
        (Events.event == "OPEN_BOOK")
        & (Events.new_value != "Chasing Sunsets - ")
        & (Events.participant_id == participant)
    )

    average_length = []
    saccades = []
    for i, book in enumerate(books):
        START_TIME_BOOK = book.time
        book_end = (
            subquery.where(Events.event == "CLOSE_BOOK")
            .where(Events.time > START_TIME_BOOK)
            .get()
        )
        END_TIME_BOOK = book_end.time

        # Get the events that have a timestamp less than the first CLOSE_BOOK event
        events_book = subquery.where(Events.time <= END_TIME_BOOK)

        # Read from participants.json whether or not the participant has low resolution data
        # These we should not divide the timestamp with 1000

        LOW_RES = json.load(open("participants.json", "r"))[f"{book.participant_id}"][
            "low_resolution"
        ]
        if not LOW_RES:
            START_TIME_BOOK /= 1000
        formatted_time = datetime.fromtimestamp(START_TIME_BOOK).strftime(
            "%Y-%m-%d_%H-%M-%S"
        )
        GAZE_FILE = f"{EYE_TRACKER_FOLDER}[{book.participant_id}]-{formatted_time}.json"
        f = open(GAZE_FILE, "r")
        GAZE_DATA_BOOK = json.load(f)
        f.close()

        timestamps = []
        x = []
        y = []

        DOMINANT_EYE = get_participant_dominant_eye(participant)

        # for each packet, plot the gaze point
        for packet in GAZE_DATA_BOOK["data"]:
            if packet[f"{DOMINANT_EYE}_gaze_point_validity"] == 0:
                continue
            x.append(
                (packet[f"{DOMINANT_EYE}_gaze_point_on_display_area"][0] * X_PIXELS)
                # * DEGREES_PER_PIXEL
            )
            y.append(
                (packet[f"{DOMINANT_EYE}_gaze_point_on_display_area"][1] * Y_PIXELS)
                # * DEGREES_PER_PIXEL
            )
            timestamps.append(packet[TIMESTAMP_IDENT])

        df = pd.DataFrame({"x": x, "y": y, "ts": timestamps})
        df = df.sort_values(by="ts")
        df = df.reset_index(drop=True)

        df["x"] = df["x"] * DEGREES_PER_PIXEL
        df["y"] = df["y"] * DEGREES_PER_PIXEL
        df["ts"] = df["ts"] / 1_000_000

        # display(df)
        SACCADIC_THRESHOLD = 80
        # Plot fixations
        fixations, v, labels = detect_fix_ivt(df, sacvel=SACCADIC_THRESHOLD)
        saccades_part = find_sacc_from_fix(fixations)
        average_length.append(np.mean(saccades_part["dxy"]))

    print(
        f"{participant} {average_length[0]} {average_length[1]} {np.std(average_length)} {average_length[0] - average_length[1]}"
    )
    avg_saccades[participant] = average_length

# print the mean saccade length for each participant
# display(df.mean())
# df.mean().to_latex("averages.tex")

    # Do a t-test to see if the mean saccade length is significantly different between the two books
    # # Perform t-test
    # t_statistic, p_value = ttest_ind(saccades[0]['len'], saccades[0]['len'])

    # # Print the results
    # print(f"T-Statistic: {t_statistic}")
    # print(f"P-Value: {p_value}")

Mean , std , difference


/Users/idk/Documents/Git/flow/apps/backend/velocityThreshold.py:62: RuntimeWarning: invalid value encountered in scalar divide
  rho = cov[0,1] / (sx*sy)


9 4.128651732126344 4.210006339748766 0.04067730381121093 -0.08135460762242186
10 6.807397256760127 4.246460198970957 1.280468528894585 2.56093705778917
11 5.5869518351881995 4.3980824672981464 0.5944346839450265 1.188869367890053
12 7.758077309793291 7.873349700973751 0.057636195590230255 -0.11527239118046051
13 4.574713033673674 3.291294480403838 0.6417092766349177 1.2834185532698354
14 6.049008409807948 5.0264375266683645 0.5112854415697918 1.0225708831395837
15 5.12264236777336 4.31681127248532 0.40291554764402004 0.8058310952880401
16 5.103884719489916 4.744307961588192 0.17978837895086208 0.35957675790172416
17 3.948171183993469 3.558468687349272 0.19485124832209832 0.38970249664419665
18 5.613232343058697 4.83691996267562 0.38815619019153846 0.7763123803830769
19 3.5662403260756337 2.9686126917030378 0.29881381718629796 0.5976276343725959
20 5.39741353885679 4.902196040032832 0.24760874941197875 0.4952174988239575
21 3.4886063445831503 3.311168735030586 0.08871880477628213 0.177

In [8]:
df = pd.DataFrame(avg_saccades)
df = df.T
df = df.rename(columns={0: 'First text', 1: 'Second text'})
# add a columnn to show the differences between book 1 and book 2
df['Difference'] =  df['First text'] - df['Second text']

# add a row to show the average of the columns
df.loc['Mean'] = df.mean()

# add a row to show the standard deviation of the columns
df.loc['Standard deviation'] = df.std()

# round all the values to 2 decimal places
df = df.round(3)

display(df)
df.to_latex('differences.tex')

,First text,Second text,Difference
9,4.129,4.210,-0.081
10,6.807,4.246,2.561
11,5.587,4.398,1.189
12,7.758,7.873,-0.115
13,4.575,3.291,1.283
14,6.049,5.026,1.023
15,5.123,4.317,0.806
16,5.104,4.744,0.360
17,3.948,3.558,0.390
18,5.613,4.837,0.776
